In [1]:
import pandas as pd
import numpy as np

In [2]:
purchase_data = 'purchase_data.csv'

In [3]:
purchase_df = pd.read_csv(purchase_data)
purchase_df.head(10)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44
5,5,Yalae81,22,Male,81,Dreamkiss,3.61
6,6,Itheria73,36,Male,169,"Interrogator, Blood Blade of the Queen",2.18
7,7,Iskjaskst81,20,Male,162,Abyssal Shard,2.67
8,8,Undjask33,22,Male,21,Souleater,1.10
9,9,Chanosian48,35,Other / Non-Disclosed,136,Ghastly Adamantite Protector,3.58


In [4]:
# Player Count

In [5]:
Total_players = len(purchase_df['SN'].unique())
Player_count_df = pd.DataFrame([{"Total Players": Total_players}])
Player_count_df


,Total Players
0,576


In [ ]:
# Purchase Analysis (Total)

In [6]:
unique_items = len(purchase_df['Item ID'].unique())
Average_price = round(purchase_df['Price'].mean(),2)
Total_purchases = len(purchase_df['Price'])
Total_revenue = purchase_df['Price'].sum()

Analysis_df = pd.DataFrame([{"Number of Unique Items" : unique_items,"Average Price" : Average_price,
                                    "Number of Purchases" : Total_purchases, "Total Revenue" : Total_revenue}])

Analysis_df["Average Price"] = Analysis_df["Average Price"].map("${:,.2f}".format)
Analysis_df["Total Revenue"] = Analysis_df["Total Revenue"].map("${:,.2f}".format)
Purchase_Analysis_df = Analysis_df[["Number of Unique Items","Average Price",
                                              "Number of Purchases","Total Revenue"]]
Purchase_Analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [ ]:
# Gender Demographics

In [7]:
sorted_df = purchase_df.drop_duplicates(['SN'],keep = 'first')
male_count = sorted_df['Gender'].tolist().count('Male')
male_percentage = round((male_count/Total_players)*100,2)
female_count = sorted_df['Gender'].tolist().count('Female')
female_percentage = round((female_count/Total_players)*100,2)
other_count = sorted_df['Gender'].tolist().count('Other / Non-Disclosed')
other_percentage = round((other_count/Total_players)*100,2)

gender_df = pd.DataFrame({"Gender" : ["Female", "Male","Other / Non-Disclosed"],
                          "Total Count": [female_count, male_count, other_count], 
                           "Percentage of Players": [female_percentage, male_percentage, other_percentage]})
gender_df.set_index("Gender",inplace = True)
gender_df = gender_df.sort_values("Total Count",ascending=False)

gender_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [ ]:
# Purchase Analysis (Gender)

In [53]:
gender_count = pd.DataFrame(purchase_df.groupby('Gender')['Gender'].count())
purchase_count = pd.DataFrame(purchase_df.groupby('Gender')['Price'].sum())

merged_df = pd.merge(gender_count,purchase_count,left_index = True, right_index = True)
merged_df.rename(columns = {'Gender': 'Purchase Count','Price' : 'Total Purchase Value'}, inplace = True)

merged_df['Average Purchase Price'] = round(merged_df['Total Purchase Value']/merged_df['Purchase Count'],2)
merged_df['Avg Total Purchase per Person'] = round(merged_df['Total Purchase Value']/gender_df['Total Count'],2)

merged_df["Average Purchase Price"] = merged_df["Average Purchase Price"].map("${:,.2f}".format)
merged_df["Total Purchase Value"] = merged_df["Total Purchase Value"].map("${:,.2f}".format)
merged_df["Avg Total Purchase per Person"] = merged_df["Avg Total Purchase per Person"].map("${:,.2f}".format)

merged_df = merged_df.sort_values("Purchase Count",ascending=False)
merged_df

,Purchase Count,Total Purchase Value,Average Purchase Price,Avg Total Purchase per Person
Gender,,,,
Male,652,"$1,967.64",$3.02,$4.07
Female,113,$361.94,$3.20,$4.47
Other / Non-Disclosed,15,$50.19,$3.35,$4.56


In [ ]:
# Age Demographics

In [9]:
bins = [0,9,14,19,24,29,34,39,100]
age_cat = ['<10','10-14','15-19','20-24','25-29','30-34','35-39','40+']
sorted_df['Age Category'] = pd.cut(sorted_df['Age'],bins = bins, labels = age_cat)
age_df = pd.DataFrame(sorted_df.groupby('Age Category')['SN'].count())
age_df.rename(columns = {"SN":"Total Count"}, inplace = True)
#del age_df['Age Category']
age_df['Percentage of Players'] = round((age_df['Total Count']/Total_players)*100,2)
age_df

/Users/madhu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Total Count,Percentage of Players
Age Category,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [ ]:
# Purchase Analysis (Age)

In [10]:
purchase_df["Age Category"] = pd.cut(purchase_df['Age'], bins = bins, labels = age_cat)
count_age = pd.DataFrame(purchase_df.groupby('Age Category')['SN'].count())
count_age.rename(columns = {"SN" : "Purchase Count"}, inplace = True)

price_count = pd.DataFrame(purchase_df.groupby('Age Category')['Price'].sum())
price_count.rename(columns = {"Price":"Total Purchase Value"}, inplace = True)

merge_count_price = pd.merge(count_age,price_count,how = 'outer',on = 'Age Category')

ave_price = round(pd.DataFrame(purchase_df.groupby('Age Category')['Price'].mean()),2)
ave_price.rename(columns={"Price":"Average Purchase Price"},inplace = True)
ave_price['Average Purchase Price']= ave_price['Average Purchase Price'].map("${:,.2f}".format)

merge_ave = pd.merge(merge_count_price,ave_price, left_index = True, right_index = True)
merge_ave['Avg Total Purchase per Person'] = round(merge_ave['Total Purchase Value']/age_df['Total Count'],2).map("${:,.2f}".format)
merge_ave["Total Purchase Value"] = merge_ave["Total Purchase Value"].map("${:,.2f}".format)
merge_ave


,Purchase Count,Total Purchase Value,Average Purchase Price,Avg Total Purchase per Person
Age Category,,,,
<10,23,$77.13,$3.35,$4.54
10-14,28,$82.78,$2.96,$3.76
15-19,136,$412.89,$3.04,$3.86
20-24,365,"$1,114.06",$3.05,$4.32
25-29,101,$293.00,$2.90,$3.81
30-34,73,$214.00,$2.93,$4.12
35-39,41,$147.67,$3.60,$4.76
40+,13,$38.24,$2.94,$3.19


In [ ]:
# Top Spenders

In [11]:
top_purchase = pd.DataFrame(purchase_df.groupby('SN')['Price'].sum())
purchase_count = pd.DataFrame(purchase_df.groupby('SN')['Price'].count())
top_purchase = top_purchase.sort_values('Price',ascending=False)

merge_top = pd.merge(top_purchase,purchase_count,how = 'outer', on='SN')
merge_top = merge_top.rename(columns={"Price_x": "Total Purchase Value","Price_y":"Purchase Count"})
merge_top['Average Purchase Price'] = round(merge_top['Total Purchase Value']/merge_top['Purchase Count'],2)

merge_top['Average Purchase Price'] = merge_top['Average Purchase Price'].map("${:,.2f}".format)
merge_top['Total Purchase Value'] = merge_top['Total Purchase Value'].map("${:,.2f}".format)
merge_top.head()

,Total Purchase Value,Purchase Count,Average Purchase Price
SN,,,
Lisosia93,$18.96,5,$3.79
Idastidru52,$15.45,4,$3.86
Chamjask73,$13.83,3,$4.61
Iral74,$13.62,4,$3.40
Iskadarya95,$13.10,3,$4.37


In [ ]:
# Most Popular Items

In [51]:
top_items_count = pd.DataFrame(purchase_df.groupby(['Item ID','Item Name'])['Price'].count())
top_items_count = top_items_count.rename(columns = {'Price': 'Purchase Count'})
top_items_count = top_items_count.sort_values('Purchase Count', ascending = False)

top_items = pd.DataFrame(purchase_df.groupby(['Item ID','Item Name'])['Price'].sum())
top_items = top_items.rename(columns={'Price' : 'Total Purchase Value'})
top_items = top_items.sort_values('Total Purchase Value',ascending = False)

summary_top = pd.merge(top_items_count,top_items,how='outer', on = ['Item ID','Item Name'])
summary_top['Item Price'] = (top_items['Total Purchase Value']/top_items_count['Purchase Count']).map("${:,.2f}".format)
summary_top['Total Purchase Value']= summary_top['Total Purchase Value'].map("${:,.2f}".format)

summary_top[['Purchase Count','Item Price','Total Purchase Value']].head()


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [ ]:
# Most Profitable Items

In [94]:
most_profitable = summary_top.sort_values(['Purchase Count','Item Price'], ascending = False)
most_profitable.head()

,,Purchase Count,Total Purchase Value,Item Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$4.23
82,Nirvana,9,$44.10,$4.90
145,Fiery Glass Crusader,9,$41.22,$4.58
108,"Extraction, Quickblade Of Trembling Hands",9,$31.77,$3.53
92,Final Critic,8,$39.04,$4.88
